# IR Project 4

## Emotion Classification

In [46]:
#Adapted from https://www.geeksforgeeks.org/emotion-classification-using-nrc-lexicon-in-python/

In [5]:
#!pip install nrclex

In [22]:
# Import libraries
import json
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from bertopic import BERTopic
import demoji
from nrclex import NRCLex 
import pandas as pd

[nltk_data] Downloading package stopwords to /Users/maga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# Load data
infile = open('tweets.json','rb')
tweets = json.load(infile)
infile.close()

In [27]:
# Function to remove stopwords
def remove_stopwords(text, lang): 
    text_wo_stopwords = []
    # remove stopwords   
    if lang == 'en':
        stop_words = set(stopwords.words('english'))
    else:
        stop_words = set(stopwords.words('spanish'))
    text = re.split('\s',text)
    for t in text:
        if t not in stop_words:
            text_wo_stopwords.append(t)
    text_wo_stopwords = ' '.join(text_wo_stopwords)
    return text_wo_stopwords

In [26]:
# Function to preprocess tweets
def preprocess(raw_tweet,lang):
    text = raw_tweet.lower() # convert to lowercase
    text = re.sub('\n',' ',text) # remove '\n'
    text = re.sub(r"http\S+", "",text) # remove urls
    text = re.sub('#',' ',text) # remove '#' but leave text from hashtag
    text = re.sub('@[a-zA-Z]+',' ', text) # remove mentions
    text = re.sub('^rt ',' ', text) # remove 'rt'
    text = re.sub('[,\.\:\!¡\?\¿\_–-\’\$%|]',' ',text) # remove punctuation
    text = re.sub('[0-9]+', ' ',text) # remove numbers
    # emojis = list(demoji.findall(text).keys()) # in case we want to store emojis
    text = demoji.replace(text, '')
    text = re.sub('\s+',' ',text) # remove extra whitespaces
    text = re.sub('^\s+','',text) # remove space(s) at start
    text = re.sub('\s+$','',text) # remove space(s) at end
    #text = re.split('\s',text) # tokenize in terms of white space
    if lang == 'en' or 'es': # no stopwrods for hindi
        text = remove_stopwords(text,lang)
    return text

In [28]:
# Extract tweets per language
def get_tweets(tweets_dic):
    
    data_en, data_es, data_hi = [], [], []
    
    for tweet in tweets:
        text = tweet['tweet_text']
        if tweet['tweet_lang'] == 'en':
            text = preprocess(text,'en')
            data_en.append(text)
        elif tweet['tweet_lang'] == 'es':
            text = preprocess(text,'es')
            data_es.append(text)
        elif tweet['tweet_lang'] == 'hi':
            text = preprocess(text,'hi')
            data_hi.append(text) 
            
    return data_en, data_es, data_hi

In [29]:
data_en, data_es, data_hi = get_tweets(tweets)

In [45]:
data_en[0]

'th pragati meeting today reviewed eight projects spread across ministries railways roads power pet…'

In [33]:
df = pd.DataFrame(data_en, columns =['Tweets'])

In [34]:
df

,Tweets
0,th pragati meeting today reviewed eight projec...
1,tomorrow th november major day india uttar pra...
2,indeed sir lachit diwas bow courageous lachit ...
3,today lachit diwas pay tributes brave lachit b...
4,"""agenda groups transcends modi targeting india..."
...,...
16050,@ walterp resist dd covid variants around time...
16051,heard yet doin' bojo amazing live uk fed track...
16052,@ lokeshsharma rajasthan coronaupdate covid ca...
16053,covid status report - - fightagainstcoronaviru...


In [41]:
# Assign emotion
i = 0
emotions = []
for tweet in data_en:
    text = ''.join(tweet)
    print(text)
    emotion = NRCLex(text)
    print(emotion.top_emotions)
    emotions.append(emotion.top_emotions)

th pragati meeting today reviewed eight projects spread across ministries railways roads power pet…
[('fear', 0.0), ('anger', 0.0), ('anticip', 0.0), ('trust', 0.0), ('surprise', 0.0), ('positive', 0.0), ('negative', 0.0), ('sadness', 0.0), ('disgust', 0.0), ('joy', 0.0)]
tomorrow th november major day india uttar pradesh strides infra creation pm foundation…
[('positive', 0.5), ('anticipation', 0.5)]
indeed sir lachit diwas bow courageous lachit borphukan outstanding leader strat…
[('positive', 0.4444444444444444)]
today lachit diwas pay tributes brave lachit borphukan widely remembered torchbearer valou…
[('trust', 0.25), ('positive', 0.25), ('joy', 0.25), ('anticipation', 0.25)]
"agenda groups transcends modi targeting indian state unity lifelong karmyogi like mo…
[('trust', 0.5), ('positive', 0.5)]
understanding pm announced withdrawal farm bills via namo app
[('trust', 0.3333333333333333), ('positive', 0.3333333333333333), ('anticipation', 0.3333333333333333)]
congratulations sir 

[('fear', 0.25), ('surprise', 0.25), ('negative', 0.25), ('sadness', 0.25)]
problem natural immunity free
[('fear', 0.3333333333333333), ('negative', 0.3333333333333333), ('sadness', 0.3333333333333333)]
experts vaccine immunity waning signs israel facing fifth covid wave "the expert cabinet advising israel's government…
[('positive', 0.6)]
masks anything immune system does-natural immunity best ivermectin zinc excellent also
[('trust', 0.5)]
problem natural immunity free
[('fear', 0.3333333333333333), ('negative', 0.3333333333333333), ('sadness', 0.3333333333333333)]
sf county fully vaxxed indoor mask mandate vax passports / k hospitalized la county fully vaxxed indoor m…
[('trust', 0.6666666666666666)]
week chapel hill enews service adjustments thanksgiving holiday nov - ; orange county indoor mask mandate continues; chapel hill scores municipal equality index; help neighbors need;
[('positive', 0.3333333333333333), ('joy', 0.3333333333333333)]
natural immunity x likely never catch c

[('positive', 0.5), ('anticipation', 0.5)]
majority stuff book media blowing wish talk positive shit book begin
[('positive', 0.2857142857142857)]
blackpink's lisa tests positive covid- jennie jisoo &amp; rosé classified come close co…
[('fear', 0.0), ('anger', 0.0), ('anticip', 0.0), ('trust', 0.0), ('surprise', 0.0), ('positive', 0.0), ('negative', 0.0), ('sadness', 0.0), ('disgust', 0.0), ('joy', 0.0)]
ch course defend jisoo toxic co breeds also worried jisoo positive job health co breeds still asking jisoo positive well tf
[('positive', 0.2857142857142857), ('negative', 0.2857142857142857)]
blackpink lisa tested positive covid- production company says group's members close…
[('positive', 0.5), ('anticipation', 0.5)]
absolutely positive greater manchester going world record messiahs performed one county th…
[('trust', 0.5), ('positive', 0.5)]
split-screen using english language articulate positive vision america t…
[('positive', 0.6666666666666666)]
kst lisa tested positive covid- a

[('positive', 0.3333333333333333)]
returning w/ sidekick got vaccinations today… greater good canine brethren health pretty simple concept eh getvaxxed
[('positive', 0.3)]
right article "in fact shows people vaccinated covid- lower death rates vaccinated even covid deaths excluded " getvaxxed
[('negative', 0.23076923076923078), ('sadness', 0.23076923076923078)]
get vaccinated get booster getvaccinated getvaccinatednow getvaxxed
[('fear', 0.0), ('anger', 0.0), ('anticip', 0.0), ('trust', 0.0), ('surprise', 0.0), ('positive', 0.0), ('negative', 0.0), ('sadness', 0.0), ('disgust', 0.0), ('joy', 0.0)]
also important sitting fence vaccines remain utterly safe age group getvaxxed
[('trust', 0.4), ('positive', 0.4)]
also important sitting fence vaccines remain utterly safe age group getvaxxed
[('trust', 0.4), ('positive', 0.4)]
thank doctors nurses thanksgiving hospitals siege covid cases rising getvaxxed
[('positive', 0.4), ('joy', 0.4)]
perfect visual getvaxxed
[('trust', 0.25), ('positive'

[('positive', 0.75)]
"we close putting covid rear view mirror" - mayor comments intimidation …
[('negative', 0.4)]
new - new milestone bc eligible people older received first dose covid- vaccine bcp…
[('positive', 0.75)]
new yorkers get first dose covid- vaccine somos locations receive first sites city-run
[('positive', 1.0)]
need first dose second dose booster dose come south central etobicoke vaccine engagement team / …
[('trust', 0.5), ('positive', 0.5)]
addition ( ) eligible adults b c received first dose ( ) received second dose
[('positive', 1.0)]
new - new milestone bc eligible people older received first dose covid- vaccine bcp…
[('positive', 0.75)]
jc five year-old son got first vaccine dose cry complain even wince is…
[('negative', 0.2727272727272727), ('sadness', 0.2727272727272727)]
new - new milestone bc eligible people older received first dose covid- vaccine bcp…
[('positive', 0.75)]
breaking germany reports new coronavirus cases far biggest one-day increase record som…


...

[('fear', 0.0), ('anger', 0.0), ('anticip', 0.0), ('trust', 0.0), ('surprise', 0.0), ('positive', 0.0), ('negative', 0.0), ('sadness', 0.0), ('disgust', 0.0), ('joy', 0.0)]
see sacked unemployed covidiot cartoon propagandist bob moran crept back obsessive toxic discourse…
[('negative', 0.4)]
weird thing pro-chinaish like wake hk rioters protestors recently displayed covidiot behaviour keeps saying leave without go china doubt
[('negative', 0.3333333333333333)]
see sacked unemployed covidiot cartoon propagandist bob moran crept back obsessive toxic discourse cost job much time hands moran reality
[('negative', 0.2857142857142857)]
ohoolahan slow news day comedy carl cardigan must unavailable usual covidiot content
[('trust', 0.4), ('positive', 0.4)]
jim jordan blasted revealing covid inform public gymjordan wednesdaywisdom wednesdaymotivation wednesdaymorning wednesdayvibe happy thanksgiving eve covid covid covidiot
[('positive', 0.3333333333333333)]
george really lost able travel asia 

[('negative', 0.3333333333333333)]
science wear mask restaurant remove sitting safe sitting covidiot helth makes perfect sense got booster
[('trust', 0.21428571428571427), ('positive', 0.21428571428571427)]
aw let nsw vaxxed covidiot
[('fear', 0.0), ('anger', 0.0), ('anticip', 0.0), ('trust', 0.0), ('surprise', 0.0), ('positive', 0.0), ('negative', 0.0), ('sadness', 0.0), ('disgust', 0.0), ('joy', 0.0)]
look covidiot domestic terrorists streets ilk supports racist one nation gunslingers cq coal miners alp dis-serviced decent australians defaming bob brown reason lnp return preferences account lnp “landslide” win
[('trust', 0.2857142857142857), ('positive', 0.2857142857142857)]
another pro trump covidiot sharing unintelligent crap think lost brain cells listening waffle freedumb
[('negative', 0.375)]
blocked could check see covidiot guessing
[('fear', 0.0), ('anger', 0.0), ('anticip', 0.0), ('trust', 0.0), ('surprise', 0.0), ('positive', 0.0), ('negative', 0.0), ('sadness', 0.0), ('disg

[('fear', 0.0), ('anger', 0.0), ('anticip', 0.0), ('trust', 0.0), ('surprise', 0.0), ('positive', 0.0), ('negative', 0.0), ('sadness', 0.0), ('disgust', 0.0), ('joy', 0.0)]
indiafightscorona lakh ( ) covid samples tested last hours together w…
[('fear', 0.0), ('anger', 0.0), ('anticip', 0.0), ('trust', 0.0), ('surprise', 0.0), ('positive', 0.0), ('negative', 0.0), ('sadness', 0.0), ('disgust', 0.0), ('joy', 0.0)]
indiafightscorona india registered thousand ( ) covid recoveries last hours toget…
[('fear', 0.0), ('anger', 0.0), ('anticip', 0.0), ('trust', 0.0), ('surprise', 0.0), ('positive', 0.0), ('negative', 0.0), ('sadness', 0.0), ('disgust', 0.0), ('joy', 0.0)]
flight/passengers movement kandla airport total flight movement arrivals- departures - del-kandla-del amd-kandla-amd arr pax - dep pax - covidappropriatebehaviour official
[('trust', 0.5), ('anticipation', 0.5)]
indiafightscorona lakh ( ) covid samples tested last hours together w…
[('fear', 0.0), ('anger', 0.0), ('anticip', 

[('surprise', 0.2), ('positive', 0.2), ('joy', 0.2), ('anticipation', 0.2)]
turner lebron available ad listed questionable likely per vogel fever returned
[('fear', 0.3333333333333333), ('negative', 0.3333333333333333), ('disgust', 0.3333333333333333)]
since pandemic i've - learned mental illness - learned cabin fever really joke bedrest
[('fear', 0.375), ('negative', 0.375)]
anthony davis likely tonight indiana said frank vogel fever returned today davis bed…
[('fear', 0.3333333333333333), ('trust', 0.3333333333333333), ('positive', 0.3333333333333333)]
charts streamed th generation kpop group song spotify fever g…
[('fear', 1.0)]
know someday im gonna meet her… fever dream
[('fear', 1.0)]
parker gives baby fever seven weeks old need another one yet
[('fear', 0.3333333333333333), ('positive', 0.3333333333333333), ('joy', 0.3333333333333333)]
anthony davis likely tonight indiana said frank vogel fever returned today davis bed…
[('fear', 0.3333333333333333), ('trust', 0.3333333333333333

[('anger', 0.2), ('negative', 0.2), ('sadness', 0.2), ('disgust', 0.2), ('anticipation', 0.2)]
thank research part goes discuss masking may inadequate done improperly wearamask getvaccinatednow enjoy passports rights also come duties… free society
[('trust', 0.16666666666666666), ('positive', 0.16666666666666666), ('negative', 0.16666666666666666), ('sadness', 0.16666666666666666), ('joy', 0.16666666666666666), ('anticipation', 0.16666666666666666)]
+ madness excluded mask-wearing *&amp;* vaxx passport they're likeliest carriers (unvaxxed school unmasked/unventilated socialising groups) &amp; panto season upon us wearamask
[('negative', 0.2857142857142857)]
khan stats another big rise daily infections week week approx covid deaths since freedom day inc…
[('trust', 0.3333333333333333)]
@ lokeshsharma rajasthan coronaupdate covid cases reported jaipur today's recovered cases distri…
[('fear', 0.0), ('anger', 0.0), ('anticip', 0.0), ('trust', 0.0), ('surprise', 0.0), ('positive', 0.0), ('

In [43]:
df['Emotions'] = emotions

In [44]:
df.head()

,Tweets,Emotions
0,th pragati meeting today reviewed eight projec...,"[(fear, 0.0), (anger, 0.0), (anticip, 0.0), (t..."
1,tomorrow th november major day india uttar pra...,"[(positive, 0.5), (anticipation, 0.5)]"
2,indeed sir lachit diwas bow courageous lachit ...,"[(positive, 0.4444444444444444)]"
3,today lachit diwas pay tributes brave lachit b...,"[(trust, 0.25), (positive, 0.25), (joy, 0.25),..."
4,"""agenda groups transcends modi targeting india...","[(trust, 0.5), (positive, 0.5)]"


In [ ]:
# Using methods to classify emotion
#print('\n', emotion.words)
#print('\n', emotion.sentences)
#print('\n', emotion.affect_list)
#print('\n', emotion.affect_dict)
#print('\n', emotion.raw_emotion_scores)
print('\n', emotion.top_emotions)
#print('\n', emotion.affect_frequencies)